In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()


# Define schema of the csv
#userSchema = StructType().add("name", "string").add("salary", "integer")

dfCSV = spark.readStream.option("sep", ";").option("header", "false").schema("name STRING,salary INT").csv("file:///home/leonardo/Desktop/text")


dfCSV.createOrReplaceTempView("salary")

total_salary = spark.sql("select name,sum(salary) from salary group by name")

query = total_salary.writeStream.outputMode("complete").format("console").start()

query.awaitTermination()



KeyboardInterrupt: 

In [45]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import TimestampType, StringType, StructType, StructField
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.streaming import StreamingContext
from operator import attrgetter



spark = SparkSession.builder.master("local").appName("Avila-stream").config("spark.some.config.option","some-value").getOrCreate()
#ssc = StreamingContext(spark, 1)


inputPath = "file:////media/leonardo/HardDisk1/8/Big\ data/Proyects/Proyectos-Big-data/Proyecto2//mycsv/"




# Crear el dataframe que leera los archivos CSV

schema = "F1 DOUBLE, F2 DOUBLE, F3 DOUBLE, F4 DOUBLE, F5 DOUBLE, F6 DOUBLE, F7 DOUBLE, F8 DOUBLE, F9 DOUBLE, F10 DOUBLE, AuthorNum DOUBLE"


streamingDF = (
  spark
    .readStream
    .schema(schema)
    .option("maxFilesPerTrigger", 1)
    .csv(inputPath)
)


spark.conf.set("spark.sql.shuffle.partitions", "2")


cols = streamingDF.columns
cols.remove("AuthorNum")
assembler = VectorAssembler(inputCols=cols,outputCol="features")

dt = DecisionTreeClassifier(labelCol="AuthorNum", featuresCol="features",maxDepth=20)


evaluator = MulticlassClassificationEvaluator(labelCol="AuthorNum",predictionCol="prediction", metricName="accuracy")

def train_df(df,epoch_id):
    print("----WORKING ON BATCH----")
    cols = df.columns
    cols.remove("AuthorNum")
    assembler = VectorAssembler(inputCols=cols,outputCol="features")
    #df = assembler.transform(df)
    
    #train, test = df.randomSplit([0.8, 0.2],seed=20)

    #dt_model = dt.fit(train)
    #dt_prediction = dt_model.transform(test)
    #dt_accuracy = evaluator.evaluate(dt_prediction)
    #print("Accuracy Score of DecisionTreeClassifier is = {}" .format(dt_accuracy))

query = streamingDF.writeStream.foreachBatch(train_df).start()   
query.awaitTermination()
    
    




Accuracy Score of DecisionTreeClassifier is = 0.9943082311733801
Accuracy Score of DecisionTreeClassifier is = 1.0


KeyboardInterrupt: 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, explode, array, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import ChiSqSelector
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression,OneVsRest
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import dayofweek
from pyspark.sql.functions import isnan, when, count, col, lit, sum
from pyspark.sql.functions import (to_date, datediff, date_format,month)
#Este es un comentario
#Se carga el conjunto de datos

spark = SparkSession.builder.master("local").appName("Avila").config("spark.some.config.option","some-value").getOrCreate()
data = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("file:///home/leonardo/Desktop/avila.csv")

#####################################################################################################
#PRIMER PUNTO
#DESCRIPCION DEL CONJUNTO DE DATOS INICIAL
#Imprime la cantidad de registros y atributos respectivamente
#print("Registros Iniciales:",data.count(),", Atributos Iniciales:",len(data.columns))

#Tipo de los atributos
data.printSchema()

# Se revisa si existen nulos en alguno de los atributos del dataset
#print("Cantidad de Nulos en cada atributo")
#print(data.select([count(when(isnan(c),c)).alias(c) for c in data.columns]).toPandas().head())

#Descripcion de los atributos
#print("Descripcion de los atributos")
#print(data.describe().select("Summary","F1","F2","F3","F4","F5").show())
#print(data.describe().select("Summary","F6","F7","F8","F9","F10").show())

#Se verifica la correlacion entre los atributos
pd = data.toPandas()
#print("Correlacion entre atributos")
#print(pd.corr())


#Distribucion del atributo clasificador
#print("Distribucion del atributo clasificador")
#data.groupby("Author").count().show()
#####################################################################################################
#COMIENZA EL SEGUNDO PUNTO
#LIMPIEZA DE LOS DATOS

#Como se puede ver en los diagramas de cajas, el atributo F2 tiene datos que son demasiado atipicos
#Estos registros se eliminaran
#print("LIMPIEZA DE LOS DATOS")
data = data.filter(data.F2<350)
#print("Datos Demasiado Atipicos de F2 Eliminados:",data.count())




#print("Conversion de atributos categoricos a numericos")
indexer = StringIndexer(inputCol="Author", outputCol="AuthorNum")
data = indexer.fit(data).transform(data)
data = data.drop('Author')
#data.groupby("AuthorNum").count().show()



#Se balancea cada categoria

A = data.filter(data.AuthorNum == 0.0).sample(fraction=0.24)
F = data.filter(data.AuthorNum == 1.0).sample(fraction=0.53)
E = data.filter(col("AuthorNum") == 2.0).withColumn("dummy", explode(array([lit(x) for x in range(1)]))).drop('dummy')
I = data.filter(col("AuthorNum") == 3.0).withColumn("dummy", explode(array([lit(x) for x in range(1)]))).drop('dummy')
X = data.filter(col("AuthorNum") == 4.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
H = data.filter(col("AuthorNum") == 5.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
G = data.filter(col("AuthorNum") == 6.0).withColumn("dummy", explode(array([lit(x) for x in range(2)]))).drop('dummy')
D = data.filter(col("AuthorNum") == 7.0).withColumn("dummy", explode(array([lit(x) for x in range(3)]))).drop('dummy')
Y = data.filter(col("AuthorNum") == 8.0).withColumn("dummy", explode(array([lit(x) for x in range(4)]))).drop('dummy')
C = data.filter(col("AuthorNum") == 9.0).withColumn("dummy", explode(array([lit(x) for x in range(8)]))).drop('dummy')
W = data.filter(col("AuthorNum") == 10.0).withColumn("dummy", explode(array([lit(x) for x in range(17)]))).drop('dummy')
B = data.filter(col("AuthorNum") == 11.0).withColumn("dummy", explode(array([lit(x) for x in range(170)]))).drop('dummy')

#Se juntan todas las categorias balanceadas
data = A.union(B).union(C).union(D).union(E).union(F).union(G).union(H).union(I).union(W).union(Y).union(X)

#print("Conjunto Balanceado")
data.groupby("AuthorNum").count().show()
#print("Numero de Registros Dataset Limpio:",data.count(),", Atributos:",len(data.columns))

#####################################################################################################
#COMIENZA PUNTO 3
#Entrenamiento de modelos:
#Modelo 1

cols=data.columns
print(cols)
cols.remove("AuthorNum")

#assembler = VectorAssembler(inputCols=cols,outputCol="features")
#data=assembler.transform(data)
data.printSchema()

#query.repartition(1).write.csv("cc_out.csv", sep='|')

#data.coalesce(1).write.csv('mycsv.csv')
